In [38]:
''' Imports '''

import pandas as pd
import polars as pl
import numpy as np
import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import percentileofscore

from prep_data import load_pbp_participation_data, load_stats_team_tendencies_offense, load_stats_team_tendencies_defense

In [39]:
''' Load Data '''

pbp_data = load_pbp_participation_data()
offense_tendencies = load_stats_team_tendencies_offense()

print(pbp_data.head().to_string())
print(offense_tendencies.head().to_string())

  play_id          game_id old_game_id home_team away_team season_type week posteam posteam_type defteam side_of_field yardline_100   game_date quarter_seconds_remaining half_seconds_remaining game_seconds_remaining game_half quarter_end drive   sp  qtr down goal_to_go   time   yrdln ydstogo ydsnet                                                                                              desc play_type yards_gained shotgun no_huddle qb_dropback qb_kneel qb_spike qb_scramble pass_length pass_location air_yards yards_after_catch run_location run_gap field_goal_result kick_distance extra_point_result two_point_conv_result home_timeouts_remaining away_timeouts_remaining timeout timeout_team td_team td_player_name td_player_id posteam_timeouts_remaining defteam_timeouts_remaining total_home_score total_away_score posteam_score defteam_score score_differential posteam_score_post defteam_score_post score_differential_post no_score_prob opp_fg_prob opp_safety_prob opp_td_prob   fg_prob safet

In [40]:
''' Constants '''

VIZ_FEATURES = ['Plays / Game', '% Pass', 'Scrambles / Game', 
                '% Plays Plays_11_Personnel',
                '% Under Center', 'ADOT', 'Avg Time to Throw', 'MaxTargetShare', 
                '% Rush Outside', 'MaxRushAttemptsShare']


# Offense

In [82]:
''' Visualize a team '''
# TODO - % rushes from under center vs shotgun

def offense_team_spider_chart(team: str, season: int):

    ## Data ##
    # Get slice from offensive tendencies
    team_sl = offense_tendencies.loc[(offense_tendencies.index.get_level_values('posteam') == team) &
                                     (offense_tendencies.index.get_level_values('season') == season), :]
    
    # Feature values
    team_feature_vals = team_sl[VIZ_FEATURES].values.tolist()[0]
    
    # Feature value percentiles
    vals_fmt = []
    pct_scores = []
    pct_scores_fmt = []
    for i in range(len(VIZ_FEATURES)):
        feature = VIZ_FEATURES[i]
        val = team_feature_vals[i]
        pct_score = percentileofscore(offense_tendencies[feature].tolist(), val, kind='weak') / 100
        
        val_fmt = f'{val:.1%}' if feature[0] == '%' else f'{val:.2f}'
        vals_fmt.append(val_fmt)
        pct_scores.append(pct_score)
        pct_scores_fmt.append(f'{pct_score:.1%}')

    ## Figure ##

    fig = make_subplots(
        rows=1, cols=2, 
        column_widths=[4,3],
        horizontal_spacing=0.1,
        specs=[[{"type": "polar"}, {"type": "domain"}]]
    )

    fig.add_trace(
        go.Scatterpolar(
            r=pct_scores,
            theta=VIZ_FEATURES,
            opacity=0.7,
            fill='toself'
        ),
        row=1, col=1
    )
    fig.update_layout(
        title_text=f"Team: {season} {team}",
        polar=dict(radialaxis_range=(0,1)),
        margin=dict(b=50, r=50, l=75, t=75)
    )

    fig.add_trace(
        go.Table(
            columnwidth=[2,1,1],
            header={
                "values": ['Component', 'Value', 'Percentile'],
            },
            cells={
                "values": [VIZ_FEATURES, vals_fmt, pct_scores_fmt]
            }
        ),
        row=1, col=2
    )

    fig.show()

offense_team_spider_chart('SEA', 2018)
offense_team_spider_chart('SEA', 2019)

In [47]:
''' Team personnel spider chart '''

PERSONNEL_COLS = ['11', '12', '13', '21', '22', 'Other']

offense_personnel = pbp_data.groupby(['posteam', 'season', 'OffensePersonnelGroup']).aggregate(
    Plays=('posteam', 'size')
)
offense_personnel['% Plays'] = offense_personnel['Plays'] / offense_personnel.groupby(level=['posteam', 'season'])['Plays'].sum()
offense_personnel['% Plays %ile'] = offense_personnel.groupby('OffensePersonnelGroup')['% Plays'].rank(pct=True, ascending=True)
offense_personnel = offense_personnel.reindex(labels=PERSONNEL_COLS, level='OffensePersonnelGroup')

print(offense_personnel.sort_values(by='% Plays', ascending=False).head().to_string())
# print(offense_personnel.loc[offense_personnel.index.get_level_values(2) == '11',:].sort_values(by='% Plays', ascending=False).to_string())

def offense_personnel_spider_chart(team: str, season: int):

    ## Data ##

    # Get slice from offensive tendencies
    team_sl = offense_personnel.loc[(offense_personnel.index.get_level_values('posteam') == team) &
                                     (offense_personnel.index.get_level_values('season') == season), :]
    
    # Personnel values
    cols = team_sl.index.get_level_values('OffensePersonnelGroup').tolist()
    vals = team_sl['% Plays'].tolist()
    percentiles = team_sl['% Plays %ile'].tolist()

    ## Figure ##

    fig = make_subplots(
        rows=1, cols=2, 
        column_widths=[4,3],
        horizontal_spacing=0.1,
        specs=[[{"type": "polar"}, {"type": "domain"}]]
    )

    spider = px.line_polar(
        r=vals,
        theta=cols,
        line_close=True,
        # line_dash='dash'
    )
    # spider = go.Scatterpolar(
    #     r=team_feature_vals,
    #     theta=PERSONNEL_COLS,
    #     opacity=0.7,
    #     fill='toself'
    # )
    for trace in spider.data:
        fig.add_trace(
            trace,
            row=1, col=1
        )

    polar_range = [0, offense_personnel['% Plays'].max() + 0.05]
    print(polar_range)
    fig.update_layout(
        title_text=f"Offensive Personnel: {season} {team}",
        polar=dict(
            radialaxis_range=polar_range
        ),
        margin=dict(b=50, r=50, l=75, t=75)
    )

    fig.add_trace(
        go.Table(
            columnwidth=[2,1,1],
            header={
                "values": ['Personnel', 'Value', 'Percentile'],
            },
            cells={
                "values": [cols, [f'{val:.1%}' for val in vals], [f'{p:.1%}' for p in percentiles]]
            }
        ),
        row=1, col=2
    )

    fig.show()


print(offense_personnel.head().to_string())

offense_personnel_spider_chart('LA', 2023)

                                      Plays   % Plays  % Plays %ile
posteam season OffensePersonnelGroup                               
LA      2023   11                      1078  0.952297      1.000000
        2018   11                       996  0.931712      0.995536
        2022   11                       931  0.921782      0.991071
CAR     2023   11                      1006  0.882456      0.986607
ATL     2024   11                       986  0.878005      0.982143
                                      Plays   % Plays  % Plays %ile
posteam season OffensePersonnelGroup                               
ARI     2018   11                       654  0.703226      0.691964
               12                       161  0.173118      0.334821
               13                        22  0.023656      0.413462
               21                        62  0.066667      0.653659
               22                        24  0.025806      0.666667
[0, 1.0022968197879858]


# Defense

In [48]:
defense_tendencies = load_stats_team_tendencies_defense()

print(defense_tendencies.head().to_string())

(224, 40)
                Games  Drives  Plays  Neutral_Down_Plays PassPlaysFaced RushPlaysFaced LightBoxPlays HeavyBoxPlays ZoneCoveragePlays ManCoveragePlays  FiveRushersPlays  SixPlusRushersPlays  Plays / Game  Drives / Game % Light Box % Heavy Box    % Zone     % Man % 5 Rushers % 6+ Rushers COVER_1 Plays COVER_2 Plays COVER_3 Plays COVER_4 Plays COVER_6 Plays COVER_1 Neutral_Down_Plays COVER_2 Neutral_Down_Plays COVER_3 Neutral_Down_Plays COVER_4 Neutral_Down_Plays COVER_6 Neutral_Down_Plays COVER_1 % Plays COVER_2 % Plays COVER_3 % Plays COVER_4 % Plays COVER_6 % Plays COVER_1 % Neutral Down Plays COVER_2 % Neutral Down Plays COVER_3 % Neutral Down Plays COVER_4 % Neutral Down Plays COVER_6 % Neutral Down Plays
defteam season                                                                                                                                                                                                                                                                   

In [77]:
''' Defense Coverage Spider Chart '''

PERSONNEL_COLS = [f'COVER_{i} % Plays' for i in [1,2,3,4,6]]
PERSONNEL_TYPES = [f'Cover {i}' for i in [1,2,3,4,6]]

def defense_coverage_spider_chart(team: str, season: int):

    ## Data ##

    # Get slice from offensive tendencies
    team_sl = defense_tendencies.loc[(defense_tendencies.index.get_level_values('defteam') == team) &
                                     (defense_tendencies.index.get_level_values('season') == season), :]
    
    # Personnel values
    cols = PERSONNEL_TYPES
    vals = team_sl[PERSONNEL_COLS].values.tolist()[0]
    # percentiles = team_sl['% Plays %ile'].tolist()

    vals_fmt = [f'{val:.1%}' for val in vals]

    ## Figure ##

    fig = make_subplots(
        rows=1, cols=2, 
        column_widths=[5,3],
        horizontal_spacing=0.15,
        specs=[[{"type": "polar"}, {"type": "domain"}]]
    )

    spider = px.line_polar(
        r=vals,
        theta=cols,
        line_close=True,
        text=vals_fmt,
        markers='marker+text'
        # line_dash='dash'
    )
    # spider = go.Scatterpolar(
    #     r=team_feature_vals,
    #     theta=PERSONNEL_COLS,
    #     opacity=0.7,
    #     fill='toself'
    # )
    for trace in spider.data:
        fig.add_trace(
            trace,
            row=1, col=1
        )

    polar_range = [0, defense_tendencies[PERSONNEL_COLS].max().max() + 0.05]
    print(polar_range)
    fig.update_traces(
        mode='lines+markers+text',
        textposition='top center'
    )
    fig.update_layout(
        title_text=f"Coverages: {season} {team}",
        polar=dict(
            radialaxis=dict(
                range=polar_range,
            ),
            angularaxis=dict(
                direction='clockwise'  # Reverses theta direction
            )
        ),
        margin=dict(b=50, r=50, l=75, t=75)
    )

    fig.add_trace(
        go.Table(
            columnwidth=[2,1,1],
            header={
                "values": ['Personnel', 'Value',]# 'Percentile'],
            },
            cells={
                "values": [cols, vals_fmt]#, [f'{p:.1%}' for p in percentiles]]
            }
        ),
        row=1, col=2
    )

    fig.show()



defense_coverage_spider_chart('PHI', 2024)

[0, 0.6644144144144145]
